## Preparation

### Libraries and Global Variables

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

random_state = 42

### Importing dataset

In [ ]:
pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 
df = pd.concat([X, y], axis=1)



## Exploratory Analysis

### Basic Information

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

### YData Profiling Report

In [ ]:
pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(df)

### Box plots 

In [ ]:
%matplotlib inline

features = df.columns[:-1]
n_rows = (len(features) + 1) // 2

plt.figure(figsize=(12, n_rows * 4))
for i, feature in enumerate(features):
    plt.subplot(n_rows, 2, i + 1)
    sns.boxplot(x='quality', y=feature, data=df)
    plt.title(f'{feature} vs Wine Quality')



plt.tight_layout()
plt.show()

### Histogram of the Output Score

In [ ]:
# Histogram of wine quality (target variable)
sns.countplot(x='quality', data=df)
plt.title('Distribution of Wine Quality')
plt.xlabel('Quality')
plt.ylabel('Frequency')
plt.show()

### Histogram of the All Features


In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import matplotlib.pyplot as plt


X = wine_quality.data.features
y = wine_quality.data.targets

df = pd.concat([X, y], axis=1)

plt.figure(figsize=(12, 10))

# Plot histogram for each input feature
df.iloc[:, :-1].hist(bins=20, figsize=(12, 10), layout=(4, 3))  # Exclude the target column

# Add a title
plt.suptitle('Histograms of Input Features', fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()


### Correlation Heatmap of Input Features

In [ ]:
# Calculate correlation matrix
correlation_matrix = df.corr()

# Set up the figure size
plt.figure(figsize=(12, 8))

# Create the heatmap
sns.heatmap(
    correlation_matrix, 
    annot=True,          # Show the correlation values
    cmap='coolwarm',     # Color palette for heatmap
    fmt='.2f',           # Format the correlation values to 2 decimal places
    linewidths=0.5,      # Add small linewidths between cells
    cbar_kws={'shrink': 0.8},  # Adjust the color bar size
    annot_kws={"size": 12},     # Increase the size of annotations for readability
    square=True          # Make the heatmap square (same width and height)
)

# Add title and labels for clarity
plt.title('Correlation Matrix of Features', fontsize=16)
plt.xlabel('Features', fontsize=14)
plt.ylabel('Features', fontsize=14)

# Show the plot
plt.show()

## Data Preprocessing

### Train Test Split

We split data into training and testing sets, with 80% of the data in the training set, and 20% in the testing dataset, using random_state=42. We stratified using y because our target data is very imbalanced.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

In [5]:
print('Training data set size:', X_train.shape[0])
print('Testing data set size:', X_test.shape[0])

Training data set size: 5197
Testing data set size: 1300


### Data Scaling

We scaled the training data using standardization for models such as ANN. This was performed after the train test split to avoid data leakage.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Training and Evaluation

### Artificial Neural Network Model

#### ANN Training & Hyperparameter Tuning

In [ ]:
# Ignore warnings to make the output cleaner
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.neural_network import MLPClassifier

y_train = np.ravel(y_train)  # reshaping to avoid warning

hidden_layers = [
    (100,),            # 1 layer with 100 nodes
    (200,),            # 1 layer with 200 nodes
    (100, 50),         # 2 layers: 100 nodes, then 50 nodes
    (200, 100),        # 2 layers: 200 nodes, then 100 nodes
    (100, 50, 25)      # 3 layers: 100 -> 50 -> 25
]

# Creating a grid of hyperparameters
param_grid = {
    'hidden_layer_sizes': hidden_layers,
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.005, 0.001],
    'max_iter': [1000],
    'solver': ['adam', 'sgd', 'lbfgs'],
}

# Creating ANN model
ann_model = MLPClassifier(random_state=random_state)



In [ ]:
# Performing Grid Search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=ann_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)


In [ ]:
# Best estimator
best_ann_model = grid_search.best_estimator_
y_pred_ann = best_ann_model.predict(X_test_scaled)

#### ANN Model Evaluation

In [ ]:
# Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_ann))
print("Classification Report:")
print(classification_report(y_test, y_pred_ann))

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_ann)
print(f"Accuracy: {accuracy:.2f}")


### K-Nearest Neighbors (KNN) Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


#### kNN Training

In [ ]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel()) # there is some error for pandas datafram
knn_pred = knn_classifier.predict(X_test)
print("K-Nearest Neighbors Classifier")
print("Confusion Matrix:\n", confusion_matrix(y_test, knn_pred))
print("\nClassification Report:\n", classification_report(y_test, knn_pred, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, knn_pred))

#### Fine tuning for KNN

In [ ]:
#fine tuning using Gridsearch
param_grid = { 'n_neighbors': [3, 5, 7, 9, 11, 13], 'weights': ['uniform', 'distance'], 'p': [1, 2] }
#the 1 and 2 are different types of distance, manhattan or euclidean
grid_search_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    #verbose=2
)
grid_search_knn.fit(X_train, y_train.iloc[:, 0].values.ravel())
best_knn = grid_search_knn.best_estimator_
print("Best Parameters for KNN:", grid_search_knn.best_params_)
y_best_pred_knn = best_knn.predict(X_test)
print("Confusion Matrix:",confusion_matrix(y_test, y_best_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_best_pred_knn, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_best_pred_knn))


### Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

param_grid = {
    'C': [0.05, 0.1, 1, 5, 10, 20, 50],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

grid_search_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid,
    cv=4,
    n_jobs=-1,
    scoring='accuracy',
    return_train_score=True,
    verbose=2,
)


grid_search_svm.fit(X_train, y_train.iloc[:, 0].values.ravel())
best_svm = grid_search_svm.best_estimator_
print("Best Parameters for SVM:", grid_search_svm.best_params_)
y_best_pred_svm = best_svm.predict(X_test)
print("Confusion Matrix:",confusion_matrix(y_test, y_best_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_best_pred_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_best_pred_svm))

heatmap = sns.heatmap(
    grid_search_svm.cv_results_['mean_test_score'].reshape(len(param_grid['C']), len(param_grid['kernel'])),
    annot=True,
    fmt=".3f",
    cmap="YlGnBu",
    xticklabels=param_grid['kernel'],
    yticklabels=param_grid['C']
)
plt.title("SVM Hyperparameter Tuning Heatmap")
plt.xlabel("Kernel")
plt.ylabel("C Value")
plt.show()

Fitting 4 folds for each of 32 candidates, totalling 128 fits
[CV] END ................................C=0.05, kernel=poly; total time=   1.5s
[CV] END ................................C=0.05, kernel=poly; total time=   1.6s
[CV] END ................................C=0.05, kernel=poly; total time=   1.7s
[CV] END .................................C=0.1, kernel=poly; total time=   1.7s
[CV] END ................................C=0.05, kernel=poly; total time=   1.8s
[CV] END .................................C=0.1, kernel=poly; total time=   1.7s
[CV] END .................................C=0.1, kernel=poly; total time=   2.0s
[CV] END .................................C=0.05, kernel=rbf; total time=   1.7s
[CV] END .................................C=0.05, kernel=rbf; total time=   1.7s
[CV] END .................................C=0.1, kernel=poly; total time=   2.0s
[CV] END .................................C=0.05, kernel=rbf; total time=   1.6s
[CV] END ................................C=2, k

NameError: name 'accuracy_score' is not defined

In [7]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

n_components = 7  # Set the number of components for PCA
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

    # Step 2: Fine-tuning using GridSearchCV for C and kernel
param_grid = {
        'C': [0.05, 0.1, 1, 5, 10, 20, 50],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

grid_search_svm = GridSearchCV(
        estimator=SVC(),
        param_grid=param_grid,
        cv=4,
        n_jobs=-1,
        scoring='accuracy',
        return_train_score=True,
    verbose=2,
)

    # Fit the grid search with PCA-transformed data
grid_search_svm.fit(X_train_pca, y_train.iloc[:, 0].values.ravel())

# Best estimator and parameters for C and kernel
best_svm = grid_search_svm.best_estimator_
best_params = grid_search_svm.best_params_

print(f"Best Parameters for SVM (C and kernel) with PCA (n_components={n_components}):", best_params)
print(f"Best Accuracy with PCA (n_components={n_components}): {grid_search_svm.best_score_:.4f}")
print(f"Best Accuracy on Test Set with PCA (n_components={n_components}): {accuracy_score(y_test, best_svm.predict(X_test_pca)):.4f}")

Fitting 4 folds for each of 28 candidates, totalling 112 fits


[CV] END .................................C=0.05, kernel=rbf; total time=   1.6s
[CV] END ................................C=0.05, kernel=poly; total time=   2.2s
[CV] END ..............................C=0.1, kernel=sigmoid; total time=   1.8s
[CV] END .................................C=0.05, kernel=rbf; total time=   1.6s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.7s
[CV] END ................................C=0.05, kernel=poly; total time=   2.4s
[CV] END ................................C=0.05, kernel=poly; total time=   2.4s
[CV] END .............................C=0.05, kernel=sigmoid; total time=   2.1s
[CV] END ..............................C=0.1, kernel=sigmoid; total time=   1.9s
[CV] END ..................................C=0.1, kernel=rbf; total time=   1.7s
[CV] END .................................C=0.05, kernel=rbf; total time=   1.8s
[CV] END ................................C=1, kernel=sigmoid; total time=   1.6s
[CV] END ...................

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
        'C': [0.05, 0.1, 1, 5, 10, 20, 50],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}
grid_search_svm = GridSearchCV(
        estimator=SVC(),
        param_grid=param_grid,
        cv=4,
        n_jobs=-1,
        scoring='accuracy',
        return_train_score=True,
        verbose=2,
)
grid_search_svm.fit(X_train_scaled, y_train.iloc[:, 0].values.ravel())
best_svm = grid_search_svm.best_estimator_
best_params = grid_search_svm.best_params_
print(f"Best Parameters for SVM (C and kernel) with StandardScaler:", best_params)
print(f"Best Accuracy with StandardScaler: {grid_search_svm.best_score_:.4f}")
print(f"Best Accuracy on Test Set with StandardScaler: {accuracy_score(y_test, best_svm.predict(X_test_scaled)):.4f}")

Fitting 4 folds for each of 28 candidates, totalling 112 fits
[CV] END ..............................C=0.05, kernel=linear; total time=   1.1s
[CV] END ..............................C=0.05, kernel=linear; total time=   1.2s
[CV] END ..............................C=0.05, kernel=linear; total time=   1.1s
[CV] END ..............................C=0.05, kernel=linear; total time=   1.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.1s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.1s
[CV] END ................................C=0.05, kernel=poly; total time=   1.4s
[CV] END ...............................C=0.1, kernel=linear; total time=   1.1s
[CV] END .................................C=0.1, kernel=poly; total time=   1.3s
[CV] END .................................C=0.1, kernel=poly; total time=   1.3s
[CV] END .................................C=0.1

### Scores

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
print("Best Parameters:", grid_search.best_params_)

best_svm_classifier = SVC(kernel=grid_search.best_params_['kernel'], C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'], random_state=random_state)
best_svm_classifier.fit(X_train_scaled, y_train.iloc[:, 0].values.ravel())
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)
print("Best SVM Classifier")
# print("\nClassification Report:\n", classification_report(y_test, y_pred_best_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred_best_svm))
print("f1 Score:", f1_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best_svm))

best_svm_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel())
y_pred_best_svm = best_svm_classifier.predict(X_test)
print("Best SVM Classifier")
# print("\nClassification Report:\n", classification_report(y_test, y_pred_best_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred_best_svm))
print("f1 Score:", f1_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best_svm))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

knn_classifier = KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)
knn_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel())
y_pred_knn_raw = knn_classifier.predict(X_test)

print("raw KNN")
print("Accuracy Score:", accuracy_score(y_test, y_pred_knn_raw))
print("f1 Score:", f1_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn_raw))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn_raw, zero_division=0))

In [ ]:
from sklearn.preprocessing import StandardScaler

#standardized
scaler = StandardScaler()
X_train_scaled_knn = scaler.fit_transform(X_train)
X_test_scaled_knn = scaler.transform(X_test)

knn_classifier_scaled = KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)
knn_classifier_scaled.fit(X_train_scaled_knn, y_train.iloc[:, 0].values.ravel())
y_pred_knn_scaled = knn_classifier_scaled.predict(X_test_scaled_knn)

print("scaled KNN")
print("Accuracy Score:", accuracy_score(y_test, y_pred_knn_scaled))
print("f1 Score:", f1_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn_scaled))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn_scaled, zero_division=0))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from Table 5 and Table 6
models = ['ANN', 'KNN', 'SVM']
metrics = ['Accuracy', 'F1', 'Precision', 'Recall']

# Raw input scores
raw_scores = {
    'Accuracy': [0.5108, 0.6215, 0.6169],
    'F1': [0.54, 0.6095, 0.5812],
    'Precision': [0.53, 0.62, 0.73],
    'Recall': [0.57, 0.6215, 0.6169]
}

# Standardized input scores
standardized_scores = {
    'Accuracy': [0.6208, 0.6623, 0.6538],
    'F1': [0.46, 0.6508, 0.6450],
    'Precision': [0.49, 0.658, 0.6481],
    'Recall': [0.51, 0.6623, 0.6538]
}

x = np.arange(len(models))
width = 0.35

# Plotting each metric separately
for metric in metrics:
    plt.figure(figsize=(8, 5))
    plt.bar(x - width/2, raw_scores[metric], width, label='Raw')
    plt.bar(x + width/2, standardized_scores[metric], width, label='Standardized')
    plt.xticks(x, models)
    plt.ylabel(metric)
    plt.title(f'{metric} Comparison: Raw vs Standardized Input')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import cycle
import numpy as np


classes = sorted(y_train.iloc[:, 0].unique())
n_classes = len(classes)
y_test_binarized = label_binarize(y_test, classes=classes)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'KNN': OneVsRestClassifier(KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)),
    'SVM': OneVsRestClassifier(SVC(C=50, kernel='rbf', gamma=1, probability=True)),
    'ANN': OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42))
}

colors = cycle(['darkorange', 'blue', 'green'])
plt.figure(figsize=(10, 8))

for (name, model), color in zip(models.items(), colors):
    model.fit(X_train_scaled, label_binarize(y_train.iloc[:, 0], classes=classes))
    y_score = model.predict_proba(X_test_scaled)

  
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'{name} (AUC = {roc_auc["micro"]:.2f})', color=color)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-Averaged ROC Curve for Multiclass Models')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()